In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTTrainer, SFTConfig 
from datasets import Dataset
from peft import LoraConfig
import pandas as pd
import torch
import os


OUTPUT_DIR = "/tmp"
WORKING_DIR = "/working/"
areas = ["MENTALCHAT", "DEPTHQA"]
area = areas[1]
HF_REPO_NAME = f"custom_eval_{area}"
DATA_PATH = f"data/synthetic/filtered_train_data_{area}.csv"
HF_TOKEN = ""

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(WORKING_DIR, exist_ok=True)

def load_data(file_path, tokenizer):
    df = pd.read_csv(file_path)
    
    def format_conversation(row):
        messages = [
            {"role": "user", "content": row['input']},
            {"role": "assistant", "content": row['output']}
        ]
        return tokenizer.apply_chat_template(messages, tokenize=False)
    
    formatted_texts = [format_conversation(row) for _, row in df.iterrows()]
    return Dataset.from_dict({"text": formatted_texts})

model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

peft_config = LoraConfig(
    r=16, 
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.05,
    bias="none"
)


training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=0.00015,
    weight_decay=0,
    fp16=True,
    gradient_checkpointing=True,
    optim="adamw_torch",
    save_strategy="epoch",
    report_to="none",
    seed=42,
    max_seq_length=1024,
)

train_dataset = load_data(DATA_PATH, tokenizer)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    peft_config=peft_config
)

trainer.train()

trainer.save_model(WORKING_DIR)
tokenizer.save_pretrained(WORKING_DIR)

os.system(f"huggingface-cli login --token {HF_TOKEN}")
os.system(f"huggingface-cli repo create {HF_REPO_NAME} --type model -y")

finetuned_model = AutoModelForCausalLM.from_pretrained(
    WORKING_DIR, 
    torch_dtype=torch.float16,
    trust_remote_code=True
)
finetuned_model.push_to_hub(HF_REPO_NAME)
tokenizer.push_to_hub(HF_REPO_NAME)